# Import packages

In [81]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
import openai
from IPython.display import display, HTML, Code

In [78]:
# %load_ext autoreload
# %autoreload 2

import LLM_Geo_Constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from LLM_Geo_kernel import Solution

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
# Case 4: Hospital_accessibility
task_name ='Hospital_accessibility'

TASK = r'''
For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map (unit: km), also show the hospital.
'''

# TASK = r'Diplay the  zipcode area in South Carolina (SC) and SC hospital locations in a  map.'

DATA_LOCATIONS = [
r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.",
r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'POINT_X' column, latitude in 'POINT_Y' column.",          
]

  
save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
# model=r"gpt-3.5-turbo"
model=r"gpt-4"
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.direct_request_prompt)

Prompt to get solution graph:

Your role: A professional Geo-information scientist and developer good at Python. 
Your task: Write a Python program to address the question or task: 
For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map (unit: km), also show the hospital.
 
Location for data you may need: 1. SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.
2. SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'POINT_X' column, latitude in 'POINT_Y' column. 
Your reply needs to meet these requirements: 
 1. DO NOT change the given variable names and paths.
2. Put your reply into a Python code block(enclosed by ```python and ```), NO explanation or conversation outside the code block.
3. 

In [80]:
direct_request_LLM_response = solution.get_direct_request_LLM_response()
exec(solution.direct_request_code)
display(Code(solution.direct_request_code, language='python'))

```python
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from geopy.distance import distance
from tqdm import tqdm
import numpy as np

SC_zip_boundary_url = "https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip"
SC_hospitals_url = "https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv"

def main():
    sc_zip_boundary = gpd.read_file(SC_zip_boundary_url)
    sc_hospitals = pd.read_csv(SC_hospitals_url)

    sc_hospitals['geometry'] = sc_hospitals.apply(lambda row: Point(row['POINT_X'], row['POINT_Y']), axis=1)
    hospitals_geo = gpd.GeoDataFrame(sc_hospitals, geometry="geometry", crs="EPSG:4326")
    
    hospitals_reproj = hospitals_geo.to_crs(sc_zip_boundary.crs)
    distances = []

    for index, zipcode in tqdm(sc_zip_boundary.iterrows(), total=len(sc_zip_boundary)):
        centroid = zipcode.geometry.centroid
        min_distance = float('inf')

  

  0%|                                                                                          | 0/424 [00:00<?, ?it/s]


TypeError: Failed to create Point instance from <POINT (-81.827 34.984)>.